In [20]:
import sys, os

# Tell Python to also look one folder up (where 'db' lives)
sys.path.append(os.path.abspath(".."))

# Optional: confirm it worked
print(" Project path added.")

 Project path added.


In [21]:
from db.connect_db import get_connection
import pandas as pd
from pathlib import Path

connection = get_connection()
cursor = connection.cursor()
print(" Connected to StudyBuddy database")


 Connected to StudyBuddy database


In [25]:
def insert_lookup_data(table_name, column_name, csv_filename):
    """
    Inserts the second column (e.g., type_name, category_name, etc.)
    from a CSV located in ../data/Raw_data into the specified lookup table.
    Auto-increment IDs are ignored.
    """
    import pandas as pd
    from pathlib import Path

    # Locate correct directory
    current_dir = Path.cwd()
    if current_dir.name == "inserts":
        data_dir = current_dir.parent / "data" / "Raw_data"
    else:
        data_dir = current_dir / "data" / "Raw_data"

    csv_path = data_dir / csv_filename

    # Load CSV
    df = pd.read_csv(csv_path)
    print(f" Loaded {len(df)} rows from {csv_path.name}")

    # Always use the second column (skip the ID column)
    if len(df.columns) < 2:
        raise ValueError(f"{csv_path.name} does not have enough columns (expected at least 2).")
    
    value_col = df.columns[1]
    values = df[value_col].dropna().unique().tolist()

    # Prepare SQL Insert
    insert_query = f"INSERT IGNORE INTO {table_name} ({column_name}) VALUES (%s)"

    inserted_count = 0
    for value in values:
        try:
            cursor.execute(insert_query, (value,))
            inserted_count += 1
        except Exception as e:
            print(f" Skipped '{value}': {e}")

    connection.commit()
    print(f" Inserted {inserted_count} '{column_name}' values into {table_name}")


In [26]:
# topic_types.csv → column = type_name
insert_lookup_data("topic_types", "type_name", "topic_types.csv")

# task_categories.csv → column = category_name
insert_lookup_data("task_categories", "category_name", "task_categories.csv")

# item_types.csv → column = type_name
insert_lookup_data("item_types", "type_name", "item_types.csv")

# mood_levels.csv → column = level_name
insert_lookup_data("mood_levels", "level_name", "mood_levels.csv")


 Loaded 10 rows from topic_types.csv
 Inserted 10 'type_name' values into topic_types
 Loaded 12 rows from task_categories.csv
 Inserted 12 'category_name' values into task_categories
 Loaded 25 rows from item_types.csv
 Inserted 25 'type_name' values into item_types
 Loaded 5 rows from mood_levels.csv
 Inserted 5 'level_name' values into mood_levels


In [27]:
for table in ["topic_types", "task_categories", "item_types", "mood_levels"]:
    cursor.execute(f"SELECT * FROM {table} LIMIT 10;")
    print(f"\n  {table}:")
    for row in cursor.fetchall():
        print(row)



  topic_types:
(4, 'Discussion')
(8, 'Group Project')
(2, 'Lab')
(1, 'Lecture')
(7, 'Practical')
(9, 'Research')
(10, 'Review')
(6, 'Seminar')
(3, 'Tutorial')
(5, 'Workshop')

  task_categories:
(1, 'Assignment')
(3, 'Exam Prep')
(7, 'Lab')
(8, 'Meeting')
(12, 'Misc Task')
(9, 'Presentation')
(2, 'Project')
(5, 'Quiz')
(6, 'Reading')
(11, 'Rest')

  item_types:
(24, 'Accessory')
(7, 'Animal')
(12, 'Apartment')
(9, 'Bird')
(15, 'Bridge')
(13, 'Building')
(4, 'Bush')
(20, 'Decoration')
(17, 'Floor')
(2, 'Flower')

  mood_levels:
(2, 'Bad')
(4, 'Good')
(5, 'Great')
(3, 'Neutral')
(1, 'Very Bad')
